In [55]:
import numpy as np
import glob, os, sys
import copy
import pandas as pd
import datetime
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select, Div
from bokeh.models import LabelSet
from bokeh.models.glyphs import MultiLine, Text
from bokeh.plotting import figure, output_file, show

In [47]:
rmms_archive_dir = '/project/MJO_GCSS/MJO_monitoring/processed_MJO_data/mogreps/rmms_new/20240109'

In [48]:
date = datetime.datetime(2024, 1, 9)
# All ensemble members
members = [str('%03d' % mem) for mem in range(36)]

In [49]:
rmm_file_names = [os.path.join(rmms_archive_dir, f'createdPCs.15sn.{date.strftime("%Y%m%d")}.fcast.{mem}.txt') for mem in members]

In [106]:
df = pd.read_csv(rmm_file_names[0])
df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')

In [130]:
def make_ens_mean_df(rmm_file_names):
    # read one file
    df = pd.read_csv(rmm_file_names[0])
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
    # Make dummy for ensemble mean
    ens_mean_df = df.copy()
    # Loop through each CSV file, read it into a DataFrame, and concatenate to the combined_df
    for col in ['rmm1', 'rmm2', 'amp']:
        ens_mean_df[col] = np.array([pd.read_csv(file)[col] for file in rmm_file_names]).mean(axis=0)
    # Phase has to be integer
    ens_mean_df['phase'] = [int(x) for x in np.array([pd.read_csv(file)['phase'] for file in rmm_file_names]).mean(axis=0)]
    return ens_mean_df

In [139]:

#ens_mean_df

In [152]:
# Set up plot
hover = HoverTool(tooltips=[
    ("Date", "@date"),
    ("RMM1", "@rmm1"),
    ("RMM2", "@rmm2"),
    ("Phase", "@phase"),
    ("Amp", "@amp"),
    ("Label", "@label"),
    ("Member", "@member")
], mode='mouse')

ens_mean_df = make_ens_mean_df(rmm_file_names)

def make_plot(title=None):
    '''
    # Generates the axes and background for the data to be plot on
    #
    :param title:
    :return:
    '''
    plot = figure(height=500, width=500, tools=["pan, reset, save, wheel_zoom, box_zoom", hover],
                     x_range=[-4, 4], y_range=[-4, 4])

    plot.title.text = title

    # Mark the 8 sectors
    x = 4
    y = 0.707107
    linewidth = 0.25
    plot.line([-x, -y], [-x, -y], line_width=0.5, line_alpha=0.6)
    plot.line([y, x], [y, x], line_width=0.5, line_alpha=0.6)
    plot.line([-x, -y], [x, y], line_width=0.5, line_alpha=0.6)
    plot.line([y, x], [-y, -x], line_width=0.5, line_alpha=0.6)
    plot.line([-x, -1], [0, 0], line_width=0.5, line_alpha=0.6)
    plot.line([1, x], [0, 0], line_width=0.5, line_alpha=0.6)
    plot.line([0, 0], [-x, -1], line_width=0.5, line_alpha=0.6)
    plot.line([0, 0], [1, x], line_width=0.5, line_alpha=0.6)

    xt, yt = 3., 1.5
    phase_marker_source = ColumnDataSource(data=dict(xt=[-xt, -yt, yt, xt, xt, yt, -yt, -xt],
                                                     yt=[-yt, -xt, -xt, -yt, yt, xt, xt, yt],
                                                     phase_labels=[str(i) for i in range(1, 9)]))
    labels = LabelSet(x='xt', y='yt', text='phase_labels', level='glyph',
                      x_offset=0, y_offset=0, source=phase_marker_source,
                      text_color='grey', text_font_size="30pt", text_alpha=0.25)

    plot.add_layout(labels)
    plot.circle([0], [0], radius=1, color="white", line_color='grey', alpha=0.6)

    phase_name_source = ColumnDataSource(dict(x=[0, 0], y=[-3.75, 3.], text=['Indian \n Ocean', 'Western \n Pacific']))
    glyph = Text(x="x", y="y", text="text", angle=0., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    phase_name_source = ColumnDataSource(dict(x=[-3.], y=[0], text=['West. Hem\n Africa']))
    glyph = Text(x="x", y="y", text="text", angle=np.pi / 2., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    phase_name_source = ColumnDataSource(dict(x=[3.], y=[0], text=['Maritime\n continent']))
    glyph = Text(x="x", y="y", text="text", angle=-np.pi / 2., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    plot.xaxis[0].axis_label = 'RMM1'
    plot.yaxis[0].axis_label = 'RMM2'

    return plot

df_analysis = df.loc[df['label']=='analysis']
plot_mog = make_plot(title=f'MOGREPS MJO Forecasts {date.strftime("%Y-%m-%d")}')
plot_mog.line('rmm1', 'rmm2', source=df_analysis, name="analysis", line_color='grey', line_width=5, line_alpha=0.8)
plot_mog.circle('rmm1', 'rmm2', source=df_analysis, name="analysis_dots", color='grey', radius=0.05,
               alpha=0.8)

for mem, rmm_file_name in enumerate(rmm_file_names):
    df = pd.read_csv(rmm_file_name)
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
    fcast_start_index = min(df.loc[df['label']=='forecast'].index)
    # connect the forecasts to the analysis
    df_forecast = df.iloc[fcast_start_index-1:]
    
    # Add member info
    df_forecast['member'] = [mem for i in range(len(df_forecast))]
    
    plot_mog.line('rmm1', 'rmm2', source=df_forecast, name="analysis", line_color='lightskyblue', line_width=2, line_alpha=0.5)
    plot_mog.circle('rmm1', 'rmm2', source=df_forecast, name="analysis_dots", color='lightskyblue', radius=0.02,
                   alpha=0.5)

# Plot ensemble mean
fcast_start_index = min(ens_mean_df.loc[ens_mean_df['label']=='forecast'].index)
# connect the forecasts to the analysis
ens_mean_df = ens_mean_df.iloc[fcast_start_index-1:]
# Add member info
ens_mean_df['member'] = ['ens_mean' for i in range(len(ens_mean_df))]

plot_mog.line('rmm1', 'rmm2', source=ens_mean_df, name="analysis", line_color='navy', line_width=5, line_alpha=0.5)
plot_mog.circle('rmm1', 'rmm2', source=ens_mean_df, name="analysis_dots", color='navy', radius=0.05,
                   alpha=0.3)

# Specify the output file
output_file("/net/home/h03/hadpx/public_html/Monitoring_dashboard/mjo/plot.html")

# Display the plot
show(plot_mog)

/var/tmp/ipykernel_20776/594923940.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast['member'] = [mem for i in range(len(df_forecast))]
/var/tmp/ipykernel_20776/594923940.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast['member'] = [mem for i in range(len(df_forecast))]
/var/tmp/ipykernel_20776/594923940.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [127]:

#rmm1s.shape

In [129]:
ens_mean_df = make_ens_mean_df(df)
ens_mean_df

,year,month,day,rmm1,rmm2,phase,amp,label
0,2023,12,1,1.226839,-0.314248,4,1.266446,analysis
1,2023,12,2,1.356864,-0.279869,4,1.385426,analysis
2,2023,12,3,1.569487,-0.021732,4,1.569637,analysis
3,2023,12,4,1.853910,0.124617,5,1.858094,analysis
4,2023,12,5,1.803461,0.267678,5,1.823217,analysis
5,2023,12,6,2.171584,0.484405,5,2.224956,analysis
6,2023,12,7,2.161734,0.828405,5,2.315026,analysis
7,2023,12,8,1.981330,1.355711,5,2.400755,analysis
8,2023,12,9,1.809174,1.761928,5,2.525371,analysis
9,2023,12,10,1.747779,1.962770,6,2.628155,analysis


In [153]:
results = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [160]:
x = [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]

In [161]:
all(x)

True